# Adversrial Game Gobang


## Requirements

1. The `limited minimax search` and `limited alpha-beta search` will only considered `limited depth` of recursion (only considers a fixed number of moves ahead), instead of searching the game state until the terminal states.

2. The `limited minimax search` and `limited alpha-beta search` will make decision based **estimation** of the non-terminal state utility, rather than evaluating the **real** terminal state utility.

3. The algorithm must alternate between maximizing and minimizing the utility at each level of the tree, as in the original minimax and alpha-beta pruning algorithms.

3. There is a valid **heuristics** method implementation in `gobang` class to estimate **non-terminal** game board utility. 

        Heuristics are rules of thumb or shortcuts that are used to solve problems quickly and efficiently. They are often used when the optimal solution is too complex or too time-consuming to find. In the context of adversarial search, heuristics are used to estimate the utility of non-terminal game states. Heuristics can be based on domain-specific knowledge, such as the value of individual pieces in a chess game, or they can be based on general principles, such as the number of open lines in a game of Go. Heuristics are not guaranteed to provide an optimal solution, but they can be very effective in practice, especially when combined with other search techniques such as alpha-beta pruning.

4. The **Agent** player, using `limited minimax search` or `limited alpha-beta search`, can play the game intelligently. => When play the `gobang` game against a human player, **Agent** will not lose the game within **20 steps**.

5. The **Agent** player, using `limited minimax search` or `limited alpha-beta search`, can play the game efficiently. => When play the `gobang` game against a human player, **Agent** make every move within **20 seconds**.



## Required libaray installation

In [1]:
import math
import random
import time
import pygame
import sys
from collections import defaultdict

pygame 2.3.0 (SDL 2.24.2, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Game board visualization

The following section of code will enable programmer interact the game board interface. You can use this code to print out the current state of the game board, as well as get input from players for their moves.

In [2]:
class Visualize:
    def __init__(self):
        pygame.init()

        # define board size
        self.Height = 15 
        self.Width = 15
        # define colors
        self.BLACK = (41, 36, 33)
        self.WHITE = (255, 245, 238)
        self.BROWN = (174, 112, 0)
        self.BLUE = (68, 206, 246)
        self.YELLOW =(210, 180, 140)

        # define board and piece position and size
        self.WINDOW_SIZE = (700, 700)
        self.screen = pygame.display.set_mode(self.WINDOW_SIZE)
        self.pieces_surface = pygame.Surface(self.WINDOW_SIZE, pygame.SRCALPHA)
        self.highlight_surface = pygame.Surface(self.WINDOW_SIZE, pygame.SRCALPHA)

        # set window title
        pygame.display.set_caption("gobang")

        # fill background
        self.screen.fill(self.YELLOW)

        # define font
        self.font = pygame.font.Font(None, 40)

        # define board and piece position and size
        self.BOARD_POSITION = (50, 50)
        self.CELL_SIZE = 40
        self.BOARD_SIZE = (self.CELL_SIZE*self.Height, self.CELL_SIZE*self.Width)
        
 
    # draw board
    def draw_board(self):
        """Draw the board"""
        for i in range(self.Height+1):
            pygame.draw.line(self.screen, self.BLACK, (self.BOARD_POSITION[0], self.BOARD_POSITION[1] + i*self.CELL_SIZE), 
                (self.BOARD_POSITION[0]+self.BOARD_SIZE[0], self.BOARD_POSITION[1] + i*self.CELL_SIZE))
        for i in range(self.Width+1):
            pygame.draw.line(self.screen, self.BLACK, (self.BOARD_POSITION[0] + i*self.CELL_SIZE, self.BOARD_POSITION[1]), 
                (self.BOARD_POSITION[0] + i*self.CELL_SIZE, self.BOARD_POSITION[1]+self.BOARD_SIZE[1]))
        pygame.display.update()

    # draw piece
    def draw_piece(self, pos, color):
        """Draw a piece on the board, highlight the latest piece"""
        # print(pos)
        self.highlight_surface.fill(pygame.Color(0,0,0,0))
        pygame.draw.circle(self.pieces_surface, color, (self.BOARD_POSITION[0]+pos[0]*self.CELL_SIZE, self.BOARD_POSITION[1]+pos[1]*self.CELL_SIZE), 15)
        pygame.draw.circle(self.highlight_surface, self.BLUE, (self.BOARD_POSITION[0]+pos[0]*self.CELL_SIZE, self.BOARD_POSITION[1]+pos[1]*self.CELL_SIZE), 15, width=3)
        self.screen.blits([(self.pieces_surface, (0,0)), (self.highlight_surface, (0,0))])
        pygame.display.update()
        
    
    # draw end screen
    def draw_end_screen(self, winner):
        """Draw the end screen"""
        if winner == 1:
            text = self.font.render("Black wins!", True, self.BLACK)
        elif winner == 2:
            text = self.font.render("White wins!", True, self.BLACK)
        else:
            text = self.font.render("Draw!", True, self.BLACK)

        # Text background
        pygame.draw.rect(self.screen, self.WHITE, (self.WINDOW_SIZE[0]/2 - text.get_width()/2 - 10, self.WINDOW_SIZE[1]/3 - text.get_height()/2 - 10, text.get_width() + 20, text.get_height() + 20))
        pygame.draw.rect(self.screen, self.BLACK, (self.WINDOW_SIZE[0]/2 - text.get_width()/2 - 10, self.WINDOW_SIZE[1]/3 - text.get_height()/2 - 10, text.get_width() + 20, text.get_height() + 20), width=3)
        # Text
        self.screen.blit(text, (self.WINDOW_SIZE[0]/2 - text.get_width()/2, self.WINDOW_SIZE[1]/3 - text.get_height()/2))
        pygame.display.update()
        
        # Restart and Quit buttons
        pygame.draw.rect(self.screen, self.WHITE, (self.WINDOW_SIZE[0]/2 - 115, self.WINDOW_SIZE[1]/3 + 50, 110, 50))
        pygame.draw.rect(self.screen, self.BLACK, (self.WINDOW_SIZE[0]/2 - 115, self.WINDOW_SIZE[1]/3 + 50, 110, 50), width=3)
        pygame.draw.rect(self.screen, self.WHITE, (self.WINDOW_SIZE[0]/2 + 5, self.WINDOW_SIZE[1]/3 + 50, 95, 50))
        pygame.draw.rect(self.screen, self.BLACK, (self.WINDOW_SIZE[0]/2 + 5, self.WINDOW_SIZE[1]/3 + 50, 95, 50), width=3)     
        
        text = self.font.render("Restart", True, self.BLACK)
        self.screen.blit(text, (self.WINDOW_SIZE[0]/2 - 110 + 50 - text.get_width()/2, self.WINDOW_SIZE[1]/3 + 50 + 25 - text.get_height()/2))
        text = self.font.render("Quit", True, self.BLACK)
        self.screen.blit(text, (self.WINDOW_SIZE[0]/2 + 55 - text.get_width()/2, self.WINDOW_SIZE[1]/3 + 50 + 25 - text.get_height()/2))
        pygame.display.update()

        # Wait for user input
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONUP:
                    mouse_x,mouse_y = pygame.mouse.get_pos()
                    if mouse_x > self.WINDOW_SIZE[0]/2 - 100 and mouse_x < self.WINDOW_SIZE[0]/2 and mouse_y > self.WINDOW_SIZE[1]/3 + 50 and mouse_y < self.WINDOW_SIZE[1]/3 + 50 + 50:
                        return 0 # restart
                    elif mouse_x > self.WINDOW_SIZE[0]/2 and mouse_x < self.WINDOW_SIZE[0]/2 + 100 and mouse_y > self.WINDOW_SIZE[1]/3 + 50 and mouse_y < self.WINDOW_SIZE[1]/3 + 50 + 50:
                        return 1 # quit

    # get mouse click position
    def get_input(self):
        """Get the mouse click position"""
        # wait for user input
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()   
                
                if event.type == pygame.MOUSEBUTTONUP:
                    mouse_x,mouse_y = pygame.mouse.get_pos()
                    # round to the nearest cell
                    x = round((mouse_x - self.BOARD_POSITION[0])/self.CELL_SIZE)
                    y = round((mouse_y - self.BOARD_POSITION[1])/self.CELL_SIZE)
                    # check if the click is on the board if not ignore
                    if x < self.BOARD_SIZE[0] and y < self.BOARD_SIZE[1]:
                        return x,y

## Game board implementation 

Similar to tic-tac-toe game, game states of `gobang` will be represented as a `Board`class object, which is a subclass of `defaultdict` that in general will consist of `{(x, y): contents}` pairs, for example `{(0, 0): 'X', (1, 1): 'O'}` might be the state of the board after two moves. Besides the contents of squares, a board also has some attributes: 
- `.to_move` to name the player whose move it is; 
- `.width` and `.height` to give the size of the board (both 3 in tic-tac-toe, but other numbers in related games);
- `.utility` to give utility value to the terminal state of the game, it is initially set as 0 by default;
- possibly other attributes, as specified by keywords. 

As a `defaultdict`, the `Board` class has a `__missing__` method, which returns `empty` for squares that have no been assigned but are within the `width` × `height` boundaries, or `off` otherwise. The class has a `__hash__` method, so instances can be stored in hash tables.

In [3]:
class Board(defaultdict):
    """A board has the player to move, a cached utility value, 
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'
    
    def __init__(self, width=16, height=16, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
        
    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

## Limited-depth Game Search Algorithms


`Limited-depth` search is a search algorithm that limits the depth of the search tree. In adversarial games, this means that the algorithm only considers a fixed number of moves ahead, instead of searching all the way to the terminal states. This can be useful when the search space is too large to search exhaustively, or when the optimal solution is not required. 

The `limited minimax search` and `limited alpha-beta search` are adaptations of the minimax and alpha-beta pruning algorithms, respectively, that limit the depth of the search tree. 


### DEPTH_LIMIT == 0 in search algorithm

In both algorithms, there exist a situation where internal methods call each other. Due to depth limitations, the search will stop when DEPTH_LIMIT = 0. At this point, the target node has been reached, and the score of the current state should be returned. The score of this non terminating state is used to determine the maximum and minimum values, or to update the values of alpha and beta, to achieve the goal of pruning branches and reducing search time.

So after the statement "if DEPTH_LIMIT == 0:", we should return the board_score for current state.

In [4]:
def limited_minimax_search(game, state, DEPTH_LIMIT = 2):
    """Search game tree to determine best move; return (value, move) pair. DEPTH_LIMIT is the depth limit of the search tree."""
    player = state.to_move

    def max_value(state, DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state) :
             return game.utility(state, player), None
        v, move = -infinity, None
        place = game.actions(state)
        for a in place:
            if game.has_neibour(a, place):  
                if DEPTH_LIMIT == 0: 
                    return game.board_score(state, a), None
                if DEPTH_LIMIT != 0:
                    v2, _ = min_value(game.result(state, a),DEPTH_LIMIT-1)
                if v2 > v:
                    v, move = v2, a
        return v, move

    def min_value(state, DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state):
             return game.utility(state, player), None
        v, move = +infinity, None
        place = game.actions(state)
        for a in place:
            if game.has_neibour(a,place):
                if DEPTH_LIMIT == 0:
                    return game.board_score(state, a), None
                if DEPTH_LIMIT != 0:
                    v2, _ = max_value(game.result(state, a),DEPTH_LIMIT-1)
                if v2 < v:
                    v, move = v2, a        
        return v, move
    v, move = max_value(state,DEPTH_LIMIT)
    return v, move

infinity = math.inf

def limited_alphabeta_search(game, state, DEPTH_LIMIT=2):
    """Search game to determine best action; use alpha-beta pruning. DEPTH_LIMIT is the depth limit of the search tree."""

    player = state.to_move

    def max_value(state, alpha, beta,DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state):
             return game.utility(state, player), None
        v, move = -infinity, None
        place = game.actions(state)
        for a in place:
            if game.has_neibour(a, place): 
                if DEPTH_LIMIT == 0:
                    return game.board_score(state, a), None
                if DEPTH_LIMIT != 0:
                    v2, _ = min_value(game.result(state, a), alpha, beta,DEPTH_LIMIT-1)
                if v2 > v:
                    v, move = v2, a
                    alpha = max(alpha, v)
                if alpha >= beta: # beta cut-off
                    return v, move
                
        return v, move

    def min_value(state, alpha, beta,DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state):
             return game.utility(state, player), None
        v, move = +infinity, None
        place = game.actions(state)
        for a in place:
            if game.has_neibour(a, place):
                if DEPTH_LIMIT == 0:
                    return game.board_score(state, a), None
                if DEPTH_LIMIT != 0:
                    v2, _ = max_value(game.result(state, a), alpha, beta,DEPTH_LIMIT-1)
                if v2 < v:
                    v, move = v2, a
                    beta = min(beta, v)
                if beta <= alpha: # alpha cut-off
                    return v, move
                
        return v, move

    v, move = max_value(state, -infinity, +infinity, DEPTH_LIMIT)
    
    return v, move

## goBang


Moves are `(x, y)` pairs denoting squares, where `(0, 0)` is the top left, and `(16, 16)` is the bottom right (on a board of size `height=width=16`). Player need to have `k=5` same pieces in the same row or coloumn or diagonal to win the game. 

### board_status method

The method checking if player has k pieces in a line.Traverse every position on the board, and only check and count when a piece of current player is placed in that position.

For each piece, perform checks in eight directions (len (directions)=8). Each two opposite directions are a group, which forms a line. The 'record' array records whether this line of pieces are blocked, 0 = life and 1 = end. The two values in the record represent the two ends of that line. In each group, explore 4 squares in each direction. If board[x,y] = '#' or is an opponent's piece, record[a]=1. If board[x,y] = player, the variable count += 1. If it is a blank space, record[a] = 0.

After checking both ends, update the values of life, one_end and both_end based on the record. After checking all eight directions, return the three values.

### get_heuristic_score method

This method uses heuristic methods to estimate non-terminal board's utility.

According to basic knowledge of Gobang, different arrangements of pieces have different importance. Therefore, the scores of current board can be calculated separately for the player and opponent, and player_score - opponent_score will be used as the standard for decision-making.

The specific scoring method is to use the board_status method to calculate the number of life, one_end and both_end for 2-5 consecutive pieces (k = 5/4/3/2). According to the goal of forming five consecutive pieces, assign score to different board arrangement.

According to the winning conditions of five consecutive pieces, the score of five consecutive pieces is the highest, with slightly lower scores for lifeFour > 0 or one_deadFour > 1, and slightly lower scores for lifeThree > 1. When the player reaches these states, it means they are likely to win. Other states have significantly lower scores compared to previous states due to their distance from the winning state.

### adjust_weight parameter in board_score method

As mentioned above, player_score - opponent_score will be the standard for decision-making. Adjust_weight is used to adjust the weight of opponent scores in this decision. The larger adjust_weight, the more inclined the player is towards defense, while the smaller adjust_weight, the more inclined the player is towards attack (i.e., think highly of player's own score or the opponent's score).

Our basic strategy is that the primary goal of the game is to prevent opponents from winning. Therefore, the opponent's score accounts for a greater proportion in decision-making. I set adjust_weight = 30 to reach this goal.

In [5]:
class goBang():
    """Play goBang on an `height` by `width` board, needing `k` in a row (colomn or diagnoal) to win.
    'X' plays first against 'O'."""

    def __init__(self, height=16, width=16, k=5):
        self.height = height
        self.width = width
        self.k = k 
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0) #initially, the utility function is 0

        # visualize the game
        self.visualize = Visualize()
        self.visualize.draw_board()

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)
    
    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        # 0 for not win, pow(1000,6) for win, -pow(1000,6) for lose
        board.utility = (0 if not win else + pow(1000,self.k+1) if player == 'X' else -pow(1000,self.k+1))      
        return board

    def utility(self, board, player):
        """Return the terminal state utility value corresponding to different players;
        pow(1000,6) for win, -pow(1000,6) for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility
    
    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)
    
    def has_neibour(self, square, place):  
        """Is this square has neibour?
        agent will not place a piece if it has no neibour pieces on the board
        this method can reduce the size of the problem by not examing pieces with no neibours"""
        
        x, y = square
        place_temp = place.copy() 

        # allow the first move to be placed as the neighbor of the center piece   
        if len(place_temp) == self.height*self.width:  # if the board is empty       
            place_temp.remove((round(self.height/2),round(self.width/2)))    
 
        for i in range(-1, 2):
            for j in range(-1, 2):
                 # identify the squares have neibour pieces
                if not ((x+i, y+j) in place_temp) and ((x+i, y+j) in self.squares):   
                    return True
        return False

    
    def board_status(self, board, player, k):
        """The method checking if player has k pieces in a line. 
         Along the line, count the number of one-side dead, two-side dead and life ending squares
         life => no piece on both ending sides of the line;
         one_dead => no piece on one ending side of the line;
         both_dead => both ending sides occupied
         """
        life = 0
        one_dead = 0
        both_dead = 0
        
        directions = [(-1, 0), (1, 0), (-1, 1), (1, -1), (0, 1), (0, -1), (1, 1), (-1, -1)]
        for row in range(self.width):
            for col in range(self.height):
                # If this is not the current player, skip it
                if board[row, col] != player:
                    continue
                j = 0
                while j < len(directions):
                    a = 0      # a represents two different directions
                    count = 1  # the number of pieces for current player in this line
                    # record[0] and record[1] respectively indicate whether there is obstruction at both ends
                    record = [0, 0]
                    # Cycle twice to determine two opposite directions
                    while a <= 1:
                        x, y = row, col
                        for i in range(4):
                            # Search four squares in this direction
                            x += directions[j][0]
                            y += directions[j][1]
                            # Call the internal __missing__ method of the Board class
                            if board[x, y] == '#':
                                # Beyond the boundary
                                record[a] = 1
                                break
                            if board[x, y] == player:
                                count += 1
                            elif board[x, y] == ('X' if player=='O' else 'O'):
                                # The opponent's piece appears
                                record[a] = 1
                                break
                            else:
                                # Blank space appears
                                record[a] = 0
                                break
                        a += 1
                        j += 1
                        
                    if count == k:
                        if record[0] == record[1] == 0:
                            # There are k consecutive pieces and neither side is blocked
                            life += 1
                        elif record[0] == record[1] == 1:
                            # There are k consecutive pieces and both sides are blocked
                            both_dead += 1
                        else:
                            # There are consecutive k pieces and only one side is blocked
                            one_dead += 1
        
        # Each method is recalculated k times
        life = int(life/k)
        one_dead = int(one_dead/k)
        both_dead = int(both_dead/k)
        
        return life, one_dead, both_dead
        
        
    def get_heuristic_score(self, board, player):
        """Estimate non-terminal board's utility, given the current player.
        Taking the 'board status' into consideration, estimation utility value as a score evaluating the posibility of the player winning the game.
        Return the socre."""
        score = 0
        
        # five pieces
        lifeFive, one_deadFive, both_deadFive = self.board_status(board, player, 5)
        if lifeFive > 0 or one_deadFive > 0 or both_deadFive > 0:
            return 1000000

        # four pieces
        lifeFour, one_deadFour, both_deadFour = self.board_status(board, player, 4)
        if lifeFour > 0 or one_deadFour > 1:
            return 900000
        elif one_deadFour == 1:
            score += 10000
        if both_deadFour > 0:
            score += 100 * both_deadFour

        # three pieces
        lifeThree, one_deadThree, both_deadThree = self.board_status(board, player, 3)
        if lifeThree > 1:
            return 800000
        elif lifeThree == 1:
            score += 10000
        if one_deadThree > 0:
            score += 10 * one_deadThree
        if both_deadThree > 0:
            score += 0
            
        # two pieces
        lifeTwo, one_deadTwo, both_deadTwo = self.board_status(board, player, 2)
        if lifeTwo > 0:
            score += 100 * lifeTwo
        if one_deadTwo > 0:
            score += 10 * one_deadTwo
        if both_deadTwo > 0:
            score += 0
        
        return score
    

    def board_score(self, board, square):
        """Utilize the heuristic method to estimate the utility of the game board
        that applying potential move specified by `squre`. 
        The new board's utility estimation takes both players into consideration. """
        
        player = board.to_move
        
        #update the board by performing the move specified by squre
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        #calculate the board estimated utility taking both 
        adjust_weight = 30  # you can try a different adjust_weight to see what will happen
        board_score = self.get_heuristic_score(board, player) - adjust_weight * self.get_heuristic_score(board, ('O' if player == 'X' else 'X'))
        return board_score
    

    def display(self, board): print(board)


def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))


## Implementation of different players playing the game

Implement how players interact with the game, and how strategies are assigned to different players. I'll represent a player as a `callable` that will be passed two arguments: `(game, state)` and will return a `move`.
The function `player` creates a player out of a search algorithm, but you can create your own players as functions, as is done with `random_player` below. You can leave the following code section unchanged.XD

In [6]:
def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    visualize = game.visualize

    while not game.is_terminal(state): 
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        visualize.draw_piece(move, visualize.BLACK if player == 'X' else visualize.WHITE)
        print('Player', player, ':', move)
        if verbose: 
            print(state)
    print('Game over: utility', game.utility(state, 'X'), 'for X')
    
    # exit or restart the GUI
    again = visualize.draw_end_screen(1 if game.utility(state, 'X') > 0 else 2 if game.utility(state, 'X') < 0 else 0)
    if again == 0:
        main()
    else:
        pygame.quit()
        sys.exit()

    return state

In [7]:
def random_player(game, state): 
    if state.to_move == 'O':
        color = "white"
    else:
        color = "black"
    (x,y)=random.choice(list(game.actions(state)))
    game.visualize.draw_piece((x,y),color)
    return (x,y)

def human_player(game, state):
    """Make a move by reading input."""
    print(state)
    print('Legal moves are', game.actions(state))
    move = None
    while move not in game.actions(state):
        move = eval(input('Your move? '))
    # vis.draw_piece(move,"black")
    print(move)
    return move

def human_mouse_player(game, state):
    """Make a move by reading input."""
    move = None
    while move not in game.actions(state):
        move= game.visualize.get_input()
    return move

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

## Playing a Game

The main function that excuting the goBang game.

In [ ]:
def main():
    #play_game(goBang(), dict(X=human_mouse_player, O=human_mouse_player), verbose=False).utility
    #play_game(goBang(), dict(X=human_mouse_player, O=player(limited_alphabeta_search)), verbose=False).utility
    #play_game(goBang(), dict(X=player(limited_alphabeta_search), O=player(limited_alphabeta_search)), verbose=False).utility
    #play_game(goBang(), dict(X=player(limited_alphabeta_search), O=player(limited_minimax_search)), verbose=False).utility

if __name__ == '__main__':
    main()